In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score


In [ ]:
#os.chdir('..')
#os.getcwd()

'/Users/Angela/Desktop/DM/final/dmt-assignment1-2025'

In [ ]:
# Load
df = pd.read_csv('cleaned_data.csv')

In [ ]:
print(df.shape)
print(df.columns)     
df.head()             

(245, 17)
Index(['TIMESTAMP', 'PROGRAM', 'ML_COURSE_NUM', 'IR_COURSE_NUM',
       'DB_COURSE_NUM', 'CHATGPT_USAGE_NUM', 'GENDER_CATEGORIZED',
       'ROOM_ESTIMATE_CLEANED', 'STRESS_LEVEL_CLEANED', 'SPORTS_HOURS_CLEANED',
       'RANDOM_NUMBER_CLEANED', 'AGE', 'BIRTHDAY_MONTH', 'BIRTHDAY_DAY',
       'BEDTIME_CLEANED', 'GOOD_DAY_CLEANED_1', 'GOOD_DAY_CLEANED_2'],
      dtype='object')


,TIMESTAMP,PROGRAM,ML_COURSE_NUM,IR_COURSE_NUM,DB_COURSE_NUM,CHATGPT_USAGE_NUM,GENDER_CATEGORIZED,ROOM_ESTIMATE_CLEANED,STRESS_LEVEL_CLEANED,SPORTS_HOURS_CLEANED,RANDOM_NUMBER_CLEANED,AGE,BIRTHDAY_MONTH,BIRTHDAY_DAY,BEDTIME_CLEANED,GOOD_DAY_CLEANED_1,GOOD_DAY_CLEANED_2
0,2025-04-01 12:17:06.560,Artificial Intelligence,1,1.0,1,1.0,male,400.0,78.0,0.0,928393.0,NaN,1.0,1.0,00:00:00,food_drink,social
1,2025-04-01 12:17:14.118,Artificial Intelligence,1,1.0,1,1.0,female,321.0,100.0,2.0,31416.0,23.0,1.0,31.0,00:30:00,weather,food_drink
2,2025-04-01 12:17:16.917,Econometrics and Operations Research,1,1.0,1,NaN,male,200.0,100.0,4.0,5.0,NaN,9.0,20.0,00:30:00,weather,food_drink
3,2025-04-01 12:17:20.718,Econometrics and Operations Research,1,0.0,0,1.0,male,350.0,60.0,6.0,37.0,21.0,10.0,17.0,23:00:00,weather,weather
4,2025-04-01 12:17:23.836,Bioinformatics and Systems Biology,1,1.0,1,1.0,male,500.0,50.0,8.0,1.0,25.0,4.0,19.0,00:00:00,other,other


In [ ]:
# target column
df['stress_bin'] = pd.cut(df['STRESS_LEVEL_CLEANED'],
                          bins=[-1, 33, 66, 100],
                          labels=['low', 'medium', 'high'])

## Feature Selection

In [6]:
df['BEDTIME_HOUR'] = pd.to_datetime(df['BEDTIME_CLEANED'], errors='coerce').dt.hour

/var/folders/5l/chc1zqqd12d1xl92ym845ntw0000gp/T/ipykernel_26937/2672469525.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['BEDTIME_HOUR'] = pd.to_datetime(df['BEDTIME_CLEANED'], errors='coerce').dt.hour


In [7]:
df['GENDER_CATEGORIZED'] = df['GENDER_CATEGORIZED'].map({'male': 0, 'female': 1})

In [8]:
df = pd.get_dummies(df, columns=['PROGRAM'])
df = pd.get_dummies(df, columns=['GOOD_DAY_CLEANED_1'])

In [9]:
df = df.astype({col: int for col in df.columns if df[col].dtype == bool})

In [12]:
df

,TIMESTAMP,ML_COURSE_NUM,IR_COURSE_NUM,DB_COURSE_NUM,CHATGPT_USAGE_NUM,GENDER_CATEGORIZED,ROOM_ESTIMATE_CLEANED,STRESS_LEVEL_CLEANED,SPORTS_HOURS_CLEANED,RANDOM_NUMBER_CLEANED,...,PROGRAM_Mathematics,PROGRAM_OTHER,GOOD_DAY_CLEANED_1_food_drink,GOOD_DAY_CLEANED_1_leisure,GOOD_DAY_CLEANED_1_mental_wellbeing,GOOD_DAY_CLEANED_1_other,GOOD_DAY_CLEANED_1_productivity,GOOD_DAY_CLEANED_1_sleep,GOOD_DAY_CLEANED_1_social,GOOD_DAY_CLEANED_1_weather
0,2025-04-01 12:17:06.560,1,1.0,1,1.0,0.0,400.0,78.0,0.0,928393.0,...,0,0,1,0,0,0,0,0,0,0
1,2025-04-01 12:17:14.118,1,1.0,1,1.0,1.0,321.0,100.0,2.0,31416.0,...,0,0,0,0,0,0,0,0,0,1
2,2025-04-01 12:17:16.917,1,1.0,1,NaN,0.0,200.0,100.0,4.0,5.0,...,0,0,0,0,0,0,0,0,0,1
3,2025-04-01 12:17:20.718,1,0.0,0,1.0,0.0,350.0,60.0,6.0,37.0,...,0,0,0,0,0,0,0,0,0,1
4,2025-04-01 12:17:23.836,1,1.0,1,1.0,0.0,500.0,50.0,8.0,1.0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,2025-04-01 12:20:46.013,1,1.0,1,NaN,NaN,NaN,0.0,NaN,NaN,...,0,0,0,0,0,1,0,0,0,0
240,2025-04-01 12:20:47.466,1,1.0,1,NaN,0.0,460.0,60.0,5.0,56.0,...,0,0,0,1,0,0,0,0,0,0
241,2025-04-01 12:21:32.688,1,0.0,1,NaN,0.0,300.0,35.0,5.0,69.0,...,0,0,0,0,0,0,0,0,0,1
242,2025-04-01 12:21:33.656,1,1.0,1,1.0,1.0,450.0,75.0,5.0,88.0,...,0,0,0,1,0,0,0,0,0,0


In [ ]:
# adds a lot of noise
df = df.drop(index=244)

In [13]:
df.isnull().sum()

TIMESTAMP                                        0
ML_COURSE_NUM                                    0
IR_COURSE_NUM                                    1
DB_COURSE_NUM                                    0
CHATGPT_USAGE_NUM                               41
GENDER_CATEGORIZED                              18
ROOM_ESTIMATE_CLEANED                            6
STRESS_LEVEL_CLEANED                             0
SPORTS_HOURS_CLEANED                             2
RANDOM_NUMBER_CLEANED                            5
AGE                                             80
BIRTHDAY_MONTH                                  17
BIRTHDAY_DAY                                    17
BEDTIME_CLEANED                                  0
GOOD_DAY_CLEANED_2                               0
stress_bin                                       0
BEDTIME_HOUR                                     0
PROGRAM_Artificial Intelligence                  0
PROGRAM_Bioinformatics and Systems Biology       0
PROGRAM_Biomedical Sciences    

In [14]:
df = df.drop(columns=['AGE', 'BIRTHDAY_MONTH', 'BIRTHDAY_DAY'])

In [15]:
# Feature List
program_columns = [col for col in df.columns if col.startswith('PROGRAM_')]
good_day_columns = [col for col in df.columns if col.startswith('GOOD_DAY_CLEANED_1_')]

cat_features = [
    'ML_COURSE_NUM',
    'IR_COURSE_NUM',
    'DB_COURSE_NUM',
    'CHATGPT_USAGE_NUM',
    'GENDER_CATEGORIZED',
    'ROOM_ESTIMATE_CLEANED',
    'SPORTS_HOURS_CLEANED',
    'RANDOM_NUMBER_CLEANED',
    'BEDTIME_HOUR',
]

features = cat_features + program_columns + good_day_columns

## Decision Tree

In [16]:
df_model = df[features + ['stress_bin']].dropna()

X = df_model[features]
y = df_model['stress_bin']

In [17]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

In [18]:
print(X.shape)
print(y.value_counts())

(186, 30)
stress_bin
low       81
medium    56
high      49
Name: count, dtype: int64


In [19]:
selected = []
best_score = 0

for feature in features:
    trial = selected + [feature]
    
    # 5-fold cross-validation
    score = cross_val_score(
        DecisionTreeClassifier(random_state=42),
        df_model[trial],
        df_model['stress_bin'],
        cv=5
    ).mean()
    
    print(f"Tested: {feature:35} → CV score: {score:.4f}")
    
    if score > best_score:
        selected.append(feature)
        best_score = score
        print(f"Added {feature}, CV accuracy improved to {score:.4f}")
    else:
        print(f"Skipped {feature}")

print("Selected features:", selected)
print("Best CV accuracy:", round(best_score, 4))

Tested: ML_COURSE_NUM                       → CV score: 0.4516
Added ML_COURSE_NUM, CV accuracy improved to 0.4516
Tested: IR_COURSE_NUM                       → CV score: 0.4354
Skipped IR_COURSE_NUM
Tested: DB_COURSE_NUM                       → CV score: 0.4033
Skipped DB_COURSE_NUM
Tested: CHATGPT_USAGE_NUM                   → CV score: 0.4248
Skipped CHATGPT_USAGE_NUM
Tested: GENDER_CATEGORIZED                  → CV score: 0.3710
Skipped GENDER_CATEGORIZED
Tested: ROOM_ESTIMATE_CLEANED               → CV score: 0.3599
Skipped ROOM_ESTIMATE_CLEANED
Tested: SPORTS_HOURS_CLEANED                → CV score: 0.4728
Added SPORTS_HOURS_CLEANED, CV accuracy improved to 0.4728
Tested: RANDOM_NUMBER_CLEANED               → CV score: nan
Skipped RANDOM_NUMBER_CLEANED
Tested: BEDTIME_HOUR                        → CV score: 0.3868
Skipped BEDTIME_HOUR
Tested: PROGRAM_Artificial Intelligence     → CV score: 0.4034
Skipped PROGRAM_Artificial Intelligence
Tested: PROGRAM_Bioinformatics and Systems B

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:978: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 144, in __call__
    score = scorer(estimator, *args, **routed_params.get(name).score)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 472, in __call__
    return estimator.score(*args, **kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/base.py", line 572, in score
    return accuracy_score(y, self.predict(X), sample_weight=sample_weight)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/tree/_classes.py", line 530, in predict
    

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

X = df_model[selected]
y = df_model['stress_bin']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)

y_pred = dt.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Final test accuracy (bottom-up features): {round(accuracy, 4)}")

Final test accuracy (bottom-up features): 0.4821


In [ ]:
print(df_model.describe())

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

X = df_model[selected]
y = df_model['stress_bin']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)

y_pred = dt.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(" Final accuracy on test set:", round(accuracy, 4))

 Final accuracy on test set: 0.4821


In [31]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

# Hyperparameter grid
param_grid = {
    'max_depth': [3, 4, 5, 6, 7],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5]
}

# Grid Search
dt = DecisionTreeClassifier(random_state=42)
grid = GridSearchCV(dt, param_grid, cv=5)
grid.fit(X_train, y_train)

# Resultaten
print("Best CV accuracy:", round(grid.best_score_, 4))
print("Best parameters:", grid.best_params_)

# Test op testset
best_model = grid.best_estimator_
y_pred = best_model.predict(X_test)
print("Final test accuracy:", round(accuracy_score(y_test, y_pred), 4))

Best CV accuracy: 0.4692
Best parameters: {'max_depth': 4, 'min_samples_leaf': 5, 'min_samples_split': 2}
Final test accuracy: 0.5179
